In [1]:
# Import the dependencies
import pandas as pd
import gmaps
import requests
# Import the API key
from config import g_key

In [2]:
# Configure gmaps to use your Google API key
gmaps.configure(api_key=g_key)

In [3]:
# Store the WeatherPy_database.csv into a DataFrame
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,San Bartolome De Tirajana,ES,27.92,-15.57,73.00,78,20,26.40,few clouds,0,0
1,1,Japura,ID,-0.32,102.35,73.87,97,93,2.98,moderate rain,0,0
2,2,Lagoa,PT,39.05,-27.98,68.65,79,0,16.28,clear sky,0,0
3,3,Upernavik,GL,72.79,-56.15,41.86,84,97,9.89,overcast clouds,0,0
4,4,Hithadhoo,MV,-0.60,73.08,85.19,70,86,9.15,overcast clouds,0,0


In [4]:
# Ask the customer to add a minimum and maximum temperature value: input deg.F preference 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# Ask the customer whether he/ she would like for it to be raining or snowing: yes or no
raining = str(input("Do you want it to be raining? (yes/no) "))
snowing = str(input("Do you want it to be snowing? (yes/no) "))

Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the temperature, raining, and snowing criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp) &
                                       (city_data_df["Rain (inches)"] == 0 if raining == "no" else city_data_df["Rain (inches)"] > 0) &
                                       (city_data_df["Snow (inches)"] == 0 if snowing == "no" else city_data_df["Snow (inches)"] > 0)
                                      ]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
4,4,Hithadhoo,MV,-0.60,73.08,85.19,70,86,9.15,overcast clouds,0,0
5,5,Cockburn Town,TC,21.46,-71.14,86.00,62,40,13.87,scattered clouds,0,0
14,14,Satwas,IN,22.53,76.68,80.78,76,95,9.17,overcast clouds,0,0
15,15,Daman,IN,20.42,72.85,81.79,82,90,11.43,light rain,0,0
16,16,Gat,IL,31.61,34.76,75.99,71,0,2.80,clear sky,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
563,563,Chalus,IR,36.66,51.42,75.20,88,90,0.89,light intensity drizzle,0,0
565,565,Colombo,LK,6.93,79.85,78.80,100,40,4.70,scattered clouds,0,0
568,568,Thinadhoo,MV,0.53,72.93,84.58,70,94,6.60,overcast clouds,0,0
569,569,Rajpipla,IN,21.87,73.50,81.39,81,75,10.67,broken clouds,0,0


In [7]:
# Verify if no null values
preferred_cities_df.count()

City_ID                201
City                   201
Country                201
Lat                    201
Lng                    201
Max Temp               201
Humidity               201
Cloudiness             201
Wind Speed             201
Current Description    201
Rain (inches)          201
Snow (inches)          201
dtype: int64

In [8]:
# Create DataFrame to store hotel names 
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [9]:
# Set paramaters to search for a hotel:
params = {"radius": 5000,
          "type": "lodging",
          "key": g_key}

In [10]:
# Print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "lodging" and our latitude and longitude
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")

# Indicate the Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
-----------------------------
Data Retrieval Complete      
-----------------------------


In [11]:
# Display the dataframe
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Hithadhoo,MV,85.19,overcast clouds,-0.60,73.08,Scoop Guest House
5,Cockburn Town,TC,86.00,scattered clouds,21.46,-71.14,Osprey Beach Hotel
14,Satwas,IN,80.78,overcast clouds,22.53,76.68,मां नर्मदा स्टाम्प रजिस्ट्री कार्यालय
15,Daman,IN,81.79,light rain,20.42,72.85,Silver Sands Beach Resort
16,Gat,IL,75.99,clear sky,31.61,34.76,רגע בנחת


In [12]:
# info_box_template variable
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

In [13]:
# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [14]:
# Marker locations
locations = hotel_df[["Lat", "Lng"]]

# Create a gmaps figure
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.6)

# Add a marker for each location and info box
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add the marker_layer to the figure
fig.add_layer(marker_layer)

# Call the figure to plot the data
fig

Figure(layout=FigureLayout(height='420px'))

In [15]:
# Create the output file (CSV)
output_data_file = "weather_data/WeatherPy_vacation.csv"

# Export the city_data_df into a CSV
hotel_df.to_csv(output_data_file, index_label="City_ID")